In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datasetsforecast.m4 import M4
from config_models import MODEL_LIST
import torch as t

from neuralforecast.losses.numpy import mae, mse, smape, mqloss

INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /tmp/tmpdzwjbzns
INFO:torch.distributed.nn.jit.instantiator:Writing /tmp/tmpdzwjbzns/_remote_module_non_scriptable.py


# Models

In [8]:
source_dataset = 'M4'
#target_dataset = 'M3'
k_shot=100
experiment_id = '20230816' # 20230816 20231108_1 20231108_1

#final_df = pd.DataFrame(columns=['model', 'smape', 'mae', 'mqloss'])
mae_df = pd.DataFrame(columns = ['dataset', 'frequency'] + MODEL_LIST)
smape_df = pd.DataFrame(columns = ['dataset', 'frequency'] + MODEL_LIST)
mqloss_df = pd.DataFrame(columns = ['dataset', 'frequency'] + MODEL_LIST)
# for target_dataset in ['M3']: # M3
#     for frequency in ['yearly','quarterly','monthly','daily']:
# for target_dataset in ['M1']: # M3
#     for frequency in ['yearly', 'quarterly','monthly']: # ,
for target_dataset in ['Tourism']: # M3
    for frequency in ['yearly', 'quarterly','monthly']: # ,
        mae_list = []
        smape_list = []
        mqloss_list = []
        for model in MODEL_LIST:
            results_df = pd.read_csv(f'./results/transferability/forecasts/{target_dataset}/{frequency}/{model}_{k_shot}_{source_dataset}_{experiment_id}.csv').reset_index(drop=True)

            smape_loss = 100*smape(results_df['y'], results_df.iloc[:,8])
            mae_loss = mae(results_df['y'], results_df.iloc[:,8])
            mq_loss = mqloss(y=results_df['y'], y_hat=results_df.iloc[:,4:13], quantiles=np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]))#/(results_df['y'].mean()+1e-5)
            mae_list.append(np.round(mae_loss,3))
            smape_list.append(np.round(smape_loss,3))
            mqloss_list.append(np.round(mq_loss,3))

        mae_df = pd.concat([mae_df, pd.DataFrame({'dataset': [target_dataset], 'frequency': [frequency], **dict(zip(MODEL_LIST, mae_list))})], ignore_index=True)
        smape_df = pd.concat([smape_df, pd.DataFrame({'dataset': [target_dataset], 'frequency': [frequency], **dict(zip(MODEL_LIST, smape_list))})], ignore_index=True)
        mqloss_df = pd.concat([mqloss_df, pd.DataFrame({'dataset': [target_dataset], 'frequency': [frequency], **dict(zip(MODEL_LIST, mqloss_list))})], ignore_index=True)

In [9]:
smape_df

,dataset,frequency,nhits_15_512,nhits_30_1024,tft_128,tft_1024,mlp_512_8,mlp_2048_32,tcn_128_3,tcn_512_5,lstm_128_3,lstm_512_5
0,Tourism,yearly,32.310,30.623,41.859,30.151,30.003,29.081,32.254,33.209,33.959,32.397
1,Tourism,quarterly,42.776,43.798,42.510,42.333,43.091,51.637,44.837,45.859,43.129,44.457
2,Tourism,monthly,61.636,62.064,62.166,61.480,61.572,68.043,63.299,61.875,64.192,65.899


In [10]:
mqloss_df

,dataset,frequency,nhits_15_512,nhits_30_1024,tft_128,tft_1024,mlp_512_8,mlp_2048_32,tcn_128_3,tcn_512_5,lstm_128_3,lstm_512_5
0,Tourism,yearly,197.780,197.464,263.563,211.671,196.190,201.568,236.217,231.759,266.611,229.391
1,Tourism,quarterly,62.527,62.829,61.748,62.118,64.764,107.287,72.620,75.013,71.653,75.887
2,Tourism,monthly,26.089,25.813,28.457,26.426,26.339,45.551,37.281,36.713,34.664,37.352


In [ ]:
# plt.plot(results_df['y'], label='y')
# plt.plot(results_df.iloc[:,8], label='y_hat')

## DeepAR

In [ ]:
source_dataset = 'M3'
#target_dataset = 'M3'
k_shot=0
experiment_id = '20231108_1' # 20230816 20231108_1

#final_df = pd.DataFrame(columns=['model', 'smape', 'mae', 'mqloss'])
mae_df = pd.DataFrame(columns = ['dataset', 'frequency'] + MODEL_LIST)
smape_df = pd.DataFrame(columns = ['dataset', 'frequency'] + MODEL_LIST)
mqloss_df = pd.DataFrame(columns = ['dataset', 'frequency'] + MODEL_LIST)

# for target_dataset in ['M3']: # M3
#     for frequency in ['yearly','quarterly', 'monthly', 'daily']: #,'monthly','daily']:
# for target_dataset in ['M1']: # M3
#     for frequency in ['yearly', 'quarterly','monthly']: # ,
for target_dataset in ['Tourism']: # M3
    for frequency in ['yearly', 'quarterly','monthly']: # ,
        mae_list = []
        smape_list = []
        mqloss_list = []
        for model in MODEL_LIST:
            results_df = pd.read_csv(f'./results/transferability/forecasts/{target_dataset}/{frequency}/{model}_{k_shot}_{source_dataset}_{experiment_id}.csv').reset_index(drop=True)

            smape_loss = 100*smape(results_df['y'], results_df.iloc[:,9])
            mae_loss = mae(results_df['y'], results_df.iloc[:,9])
            mq_loss = mqloss(y=results_df['y'], y_hat=results_df.iloc[:,5:14], quantiles=np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]))#/(results_df['y'].mean()+1e-5)
            mae_list.append(np.round(mae_loss,3))
            smape_list.append(np.round(smape_loss,3))
            mqloss_list.append(np.round(mq_loss,3))

        mae_df = pd.concat([mae_df, pd.DataFrame({'dataset': [target_dataset], 'frequency': [frequency], **dict(zip(MODEL_LIST, mae_list))})], ignore_index=True)
        smape_df = pd.concat([smape_df, pd.DataFrame({'dataset': [target_dataset], 'frequency': [frequency], **dict(zip(MODEL_LIST, smape_list))})], ignore_index=True)
        mqloss_df = pd.concat([mqloss_df, pd.DataFrame({'dataset': [target_dataset], 'frequency': [frequency], **dict(zip(MODEL_LIST, mqloss_list))})], ignore_index=True)

In [ ]:
smape_df

In [ ]:
mqloss_df

## Wiki

In [ ]:
source_dataset = 'wiki'
#target_dataset = 'M3'
k_shot=0
experiment_id = '20231108_1'

#final_df = pd.DataFrame(columns=['model', 'smape', 'mae', 'mqloss'])
mae_df = pd.DataFrame(columns = ['dataset', 'frequency'] + MODEL_LIST)
smape_df = pd.DataFrame(columns = ['dataset', 'frequency'] + MODEL_LIST)
mqloss_df = pd.DataFrame(columns = ['dataset', 'frequency'] + MODEL_LIST)

for target_dataset in ['M3']: # , 'AirPassengers', 'ILI', 'TrafficL'
    for frequency in ['daily']: # 'yearly','quarterly','monthly',
        mae_list = []
        smape_list = []
        mqloss_list = []
        for model in MODEL_LIST:
            results_df = pd.read_csv(f'./results/transferability/forecasts/{target_dataset}/{frequency}/{model}_{k_shot}_{source_dataset}_{experiment_id}.csv').reset_index(drop=True)

            smape_loss = 100*smape(results_df['y'], results_df.iloc[:,8])
            mae_loss = mae(results_df['y'], results_df.iloc[:,8])
            mq_loss = mqloss(y=results_df['y'], y_hat=results_df.iloc[:,4:13], quantiles=np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]))
            mae_list.append(np.round(mae_loss,3))
            smape_list.append(np.round(smape_loss,3))
            mqloss_list.append(np.round(mq_loss,3))

        mae_df = pd.concat([mae_df, pd.DataFrame({'dataset': [target_dataset], 'frequency': [frequency], **dict(zip(MODEL_LIST, mae_list))})], ignore_index=True)
        smape_df = pd.concat([smape_df, pd.DataFrame({'dataset': [target_dataset], 'frequency': [frequency], **dict(zip(MODEL_LIST, smape_list))})], ignore_index=True)
        mqloss_df = pd.concat([mqloss_df, pd.DataFrame({'dataset': [target_dataset], 'frequency': [frequency], **dict(zip(MODEL_LIST, mqloss_list))})], ignore_index=True)

In [ ]:
smape_df

In [ ]:
mqloss_df

In [ ]:
source_dataset = 'wiki'
#target_dataset = 'M3'
k_shot=0
experiment_id = '20231108_1'

#final_df = pd.DataFrame(columns=['model', 'smape', 'mae', 'mqloss'])
mae_df = pd.DataFrame(columns = ['dataset', 'frequency'] + MODEL_LIST)
smape_df = pd.DataFrame(columns = ['dataset', 'frequency'] + MODEL_LIST)
mqloss_df = pd.DataFrame(columns = ['dataset', 'frequency'] + MODEL_LIST)

for target_dataset in ['M3']: # , 'AirPassengers', 'ILI', 'TrafficL'
    for frequency in ['daily']: # 'yearly','quarterly','monthly',
        mae_list = []
        smape_list = []
        mqloss_list = []
        for model in MODEL_LIST:
            results_df = pd.read_csv(f'./results/transferability/forecasts/{target_dataset}/{frequency}/{model}_{k_shot}_{source_dataset}_{experiment_id}.csv').reset_index(drop=True)

            smape_loss = 100*smape(results_df['y'], results_df.iloc[:,9])
            mae_loss = mae(results_df['y'], results_df.iloc[:,9])
            mq_loss = mqloss(y=results_df['y'], y_hat=results_df.iloc[:,5:14], quantiles=np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]))#/(results_df['y'].mean()+1e-5)
            mae_list.append(np.round(mae_loss,3))
            smape_list.append(np.round(smape_loss,3))
            mqloss_list.append(np.round(mq_loss,3))

        mae_df = pd.concat([mae_df, pd.DataFrame({'dataset': [target_dataset], 'frequency': [frequency], **dict(zip(MODEL_LIST, mae_list))})], ignore_index=True)
        smape_df = pd.concat([smape_df, pd.DataFrame({'dataset': [target_dataset], 'frequency': [frequency], **dict(zip(MODEL_LIST, smape_list))})], ignore_index=True)
        mqloss_df = pd.concat([mqloss_df, pd.DataFrame({'dataset': [target_dataset], 'frequency': [frequency], **dict(zip(MODEL_LIST, mqloss_list))})], ignore_index=True)

In [ ]:
smape_df

In [ ]:
mqloss_df

## Time

In [ ]:
source_dataset = 'M4'
k_shot = 0
experiment_id = '20231108_1'

for target_dataset in ['M3', 'M1', '']: # , 'AirPassengers', 'ILI', 'TrafficL'
time_dir = f'{time_dir}/{args.model}_{args.k_shot}_{args.source_dataset}_{args.experiment_id}.csv'

## Baselines

In [ ]:
experiment_id = '20230816_level'
BASELINES = ['naive', 'autoarima'] # 'autoarima', 'arima'
parse_dict = {'naive':'Naive', 'autoarima': 'AutoARIMA'}
#final_df = pd.DataFrame(columns=['model', 'smape', 'mae', 'mqloss'])
mae_df = pd.DataFrame(columns = ['dataset', 'frequency'] + BASELINES)
smape_df = pd.DataFrame(columns = ['dataset', 'frequency'] + BASELINES)
mqloss_df = pd.DataFrame(columns = ['dataset', 'frequency'] + BASELINES)

# for target_dataset in ['M3']: # , 'AirPassengers', 'ILI', 'TrafficL'
#     for frequency in ['yearly','quarterly','monthly','daily']:
# for target_dataset in ['Tourism']: # M3
#     for frequency in ['yearly','quarterly','monthly']:
for target_dataset in ['M1']: # M3
    for frequency in ['yearly','quarterly','monthly']:
        mae_list = []
        smape_list = []
        mqloss_list = []
        for model in BASELINES:
            results_df = pd.read_csv(f'./results/transferability/forecasts/{target_dataset}/{frequency}/{model}_{experiment_id}.csv').reset_index(drop=True)
            name = parse_dict[model]
            results_df = results_df[['ds','cutoff','y', f'{name}-lo-80', f'{name}-lo-60', f'{name}-lo-40', f'{name}-lo-20', \
                                    name, f'{name}-hi-20', f'{name}-hi-40', f'{name}-hi-60', f'{name}-hi-80']]

            smape_loss = 100*smape(results_df['y'], results_df.loc[:,name])
            mae_loss = mae(results_df['y'], results_df.loc[:,name])

            mq_loss = mqloss(y=results_df['y'], y_hat=results_df.iloc[:,3:12], quantiles=np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]))
            mae_list.append(np.round(mae_loss, 3))
            smape_list.append(np.round(smape_loss, 3))
            mqloss_list.append(np.round(mq_loss,3))

        mae_df = pd.concat([mae_df, pd.DataFrame({'dataset': [target_dataset], 'frequency': [frequency], **dict(zip(BASELINES, mae_list))})], ignore_index=True)
        smape_df = pd.concat([smape_df, pd.DataFrame({'dataset': [target_dataset], 'frequency': [frequency], **dict(zip(BASELINES, smape_list))})], ignore_index=True)
        mqloss_df = pd.concat([mqloss_df, pd.DataFrame({'dataset': [target_dataset], 'frequency': [frequency], **dict(zip(BASELINES, mqloss_list))})], ignore_index=True)

In [ ]:
smape_df

In [ ]:
mqloss_df

# Automatico

In [51]:
master_config = {
                 'mlp': {'parameters':['512_8','2048_32'], 'experiment_id':'20230816'},
                 'lstm': {'parameters':['128_3','512_5'], 'experiment_id':'20230816'},
                 'tcn': {'parameters':['128_3','512_5'], 'experiment_id':'20230816'},
                 'deepar': {'parameters':['64_2'], 'experiment_id':'20231108_1'},
                 'tft': {'parameters':['128','1024'], 'experiment_id':'20230816'},
                 'vanillatransformer': {'parameters':['256_3'], 'experiment_id':'20230816'},
                 'nhits': {'parameters':['15_512','30_1024'], 'experiment_id':'20230816'},
                 'patchtst': {'parameters':['5'], 'experiment_id':'20231108_1'},
}

dataset_config = {'M3': ['M1', 'Tourism'], 'M4': ['M1', 'M3', 'Tourism'], 'wiki': ['M3_daily']}

target_freq_config = {'M1': ['yearly', 'quarterly', 'monthly'], 'M3': ['yearly', 'quarterly', 'monthly','daily'], 'Tourism': ['yearly', 'quarterly', 'monthly'], 'M3_daily': ['daily']}

In [52]:
k_shot = 100
smape_df = pd.DataFrame(columns = ['dataset', 'frequency'] + list(master_config.keys()))

for source_dataset in dataset_config.keys(): 
    for target_dataset in dataset_config[source_dataset]:
        for frequency in target_freq_config[target_dataset]:
            if target_dataset == 'M3_daily':
                target_dataset = 'M3'
            smape_list = []
            for model in master_config.keys():
                best_smape = 1000
                for parameter in master_config[model]['parameters']:
                    experiment_id = master_config[model]['experiment_id']
                    results_df = pd.read_csv(f'./results/transferability/forecasts/{target_dataset}/{frequency}/{model}_{parameter}_{k_shot}_{source_dataset}_{experiment_id}.csv').reset_index(drop=True)
                    if model == 'deepar':
                        smape_loss = 100*smape(results_df['y'], results_df.iloc[:,9])
                    else:
                        smape_loss = 100*smape(results_df['y'], results_df.iloc[:,8])
                    if smape_loss < best_smape:
                        best_smape = smape_loss
                smape_list.append(np.round(best_smape,3))

            smape_df = pd.concat([smape_df, pd.DataFrame({'dataset': [target_dataset], 'frequency': [frequency], **dict(zip(master_config.keys(), smape_list))})], ignore_index=True)

In [53]:
smape_df

,dataset,frequency,mlp,lstm,tcn,deepar,tft,vanillatransformer,nhits,patchtst
0,M1,yearly,15.701,32.413,26.543,25.532,16.019,63.184,17.483,19.327
1,M1,quarterly,16.184,23.009,20.751,18.280,16.767,19.099,16.885,15.710
2,M1,monthly,14.571,15.686,15.226,17.171,14.390,13.960,13.816,13.737
3,Tourism,yearly,37.768,31.983,31.160,31.451,27.981,121.573,31.974,33.633
4,Tourism,quarterly,43.409,48.707,46.066,49.606,42.126,95.946,42.878,44.716
5,Tourism,monthly,61.147,64.564,62.576,68.324,61.716,62.639,60.908,63.979
6,M1,yearly,16.045,31.209,26.379,22.291,15.814,19.900,16.214,18.846
7,M1,quarterly,16.343,18.155,18.801,17.954,15.292,16.750,16.026,15.423
8,M1,monthly,13.335,15.541,15.390,17.634,13.439,13.133,13.324,13.396
9,M3,yearly,15.757,23.205,21.588,20.987,15.779,22.584,16.756,15.941
